Scraping data from newsdata.io

In [1]:
import requests
from tqdm.auto import tqdm
import json
import pandas as pd

In [10]:
API_KEY = "pub_1269702a2810da439061f0313354b3c2282fa"
N_PAGE = 2

In [4]:
news = []
for page_number in tqdm(range(2)):
    page_number = page_number + 1
    r = requests.get(f"https://newsdata.io/api/1/news?apikey=pub_1269702a2810da439061f0313354b3c2282fa&q=jewelry&language=en&page={page_number}")
    if r.status_code == 200:
        data = r.json().get('results', [])
        print(f"page {page_number} => has {len(data)} news")
        news += data
    else:
        print(f"error at page : {page_number}, status {r}")

page 1 => has 10 news
page 2 => has 10 news



In [5]:
print(f"Total news : {len(news)}")

Total news : 20


Dumb data to json file

In [6]:
with open('jewelrydata.json', 'w') as f:
    json.dump(news, f)

Upload data to DataPlatform

In [7]:
from dsmlibrary.datanode import DataNode

ModuleNotFoundError: No module named 'dsmlibrary'

In [ ]:
dir_raw_id = 45

In [8]:
datanode = DataNode()

NameError: name 'DataNode' is not defined

In [9]:
datanode.upload_file(directory_id=dir_raw_id, file_path='jewelrydata.json')

NameError: name 'datanode' is not defined

Convert json to dataframe 

In [ ]:
from tqdm.auto import tqdm
import json
import pandas as pd
from dsmlibrary.datanode import DataNode

In [ ]:
dir_raw_id = 45

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='jewelrydata.json'))
meta

In [ ]:
data = json.load(fp)
data

In [ ]:
df = pd.DataFrame(data)
df

Cleansing data and convert to CSV

In [ ]:
df2 = df.drop(columns=['title','link','video_url','creator','description','content','image_url','source_id','language'])
df2

In [ ]:
df2.rename(columns={'pubDate':'published_date','country':'country_name','category':'category_name'},inplace=True)
df2

In [ ]:
df3 = df2.drop([4,5,10,11,18])
df3

In [1]:
df3.to_csv('jewelrycleaned.csv',index=False)

NameError: name 'df3' is not defined

In [ ]:
datanode.upload_file(directory_id=dir_raw_id, file_path='jewelrycleaned.csv')

Getting uploaded file with 1NF from Dataplatform

In [ ]:
from dsmlibrary.datanode import DataNode
import json
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
dir_raw_id = 45
dir_process_id = 123

In [ ]:
datanode = DataNode()

In [ ]:
meta, fp = datanode.get_file(datanode.get_file_id(directory_id=dir_raw_id, name='jewelry1nf.csv'))
meta

In [ ]:
df = pd.read_csv(fp)
df

Covert CSV file to parquet and upload to Dataplatform

In [ ]:
datanode.write(df=df, directory=dir_process_id, name="jewelry", profiling=True, lineage=[datanode.get_file_id(directory_id=dir_raw_id, name='jewelry1nf.csv')])